In [ ]:
!pip install sacremoses

In [ ]:
import os
import glob
import re
import json
import string
import itertools


from sacremoses import MosesTokenizer, MosesDetokenizer

# -------------------HERE------------------

# # Run this for me
INPUT_FILE = "/kaggle/input/transcripts/Bdb001.interaction.txt"

# OUTPUT_DIR = "/kaggle/working/preprocessed_data/"

# OUTPUT_FILE = "preprocessed_trunscript"

# Run this for you

BASE_PATH = os.getcwd()
# INPUT_FILE = os.path.join(BASE_PATH, 'transcripts', 'Bdb001.interaction.txt')
OUTPUT_DIR = os.path.join(BASE_PATH, 'preprocessed_data')
OUTPUT_FILE = 'preprocessed_trunscript'
# -----------------------------------------

In [ ]:
def read_transcripts(meetings_file):
    transcript = ""

    with open(meetings_file, "r", encoding="utf-8") as f:
        transcript = f.read().splitlines()

    return transcript

In [ ]:
# transcript = read_transcripts(INPUT_FILE)
transcript = """المتحدث 1: مهلا ، في طريقي إلى المكتب.هل يمكنك أن ترسل لي هذا جدول الأعمال لهذا اليوم؟
المتحدث 2: بالتأكيد.إنه جاهز فقط
المتحدث 1: شكرا.هل كيتلين هناك حتى الآن؟
المتحدث 2: لا ، ليس بعد
المتحدث 1: حسنًا ، فقط أتساءل.لدينا مكالمة جماعية في التاسعة.
المتحدث 2: هل هذا مع مؤسسة الكرم؟
المتحدث 1: نعم ، إنهم يفكرون في طلب المنح لدينا ، لكنهم أرادوا إجراء مقابلة معنا أولاً
المتحدث 2: حسنًا ، حظًا سعيدًا في ذلك
المتحدث 1: شكرا.
المتحدث 2: أرسل للتو جدول أعمال.
المتحدث 1: حسنًا.سأخبرك عندما أحصل عليه.هل يمكنك التحقق مما إذا كان مارتن هناك؟أنا بحاجة إلى رد على بريد إلكتروني أرسلته إليه
المتحدث 2: إنه على الهاتف ، لكنني سأتحدث معه في أقرب وقت
المتحدث 1: شكرا.ذكّرني: هل حفلة عيد الميلاد غدًا؟
المتحدث 2: تم تأجيله إلى الخميس.أرسلنا سكوت جميعًا تحديثًا.
المتحدث 1: هذا صحيح.لا أعرف أين كانت عقلي هذا الأسبوع
المتحدث 2: أعتقد أنني أسمع مارتن يختتم دعوته الآن.
المتحدث 1: حسنًا ، عظيم.
المتحدث 2: تحدثت معه للتو.إنه يرسل لك البريد الإلكتروني الآن
المتحدث 1: شكرا رجل.فقط حصلت على جدول الأعمال أيضا.سأراك قليلا.
المتحدث 2: أراك قليلا
"""

lines = transcript.strip().split("\n")




# en_automin2023 = read_transcripts(os.path.join(ELITR_AUTOMIN_2023_DIR, AUTOMIN_EN_DIR))


In [ ]:
transcript

In [ ]:
!pip install camel_tools

In [ ]:
import re
import string
import itertools
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.utils.normalize import normalize_unicode, normalize_alef_maksura_ar, normalize_teh_marbuta_ar, normalize_alef_ar

# Parse Arabic transcript by speaker (same logic)
def parse_transcript_by_speaker(transcript):
    parsed_transcript = []

    for line in transcript:
        match = re.match(r"^المتحدث\s+(\d+):\s*(.*)", line)
        if match:
            speaker_id, utterance = match.group(1), match.group(2).strip()
            parsed_transcript.append({
                "role": f"المتحدث {speaker_id}",
                "utterance": [utterance]
            })
        elif len(parsed_transcript) > 0:
            parsed_transcript[-1]["utterance"].append(line.strip())

    return parsed_transcript

# Arabic ASR filler words (customize more as needed)
def remove_asr_errors(tokens):
    AR_STOPWORDS = [
        r"^اه+$", r"^ا+مم+$", r"^هم+م+$", r"^ا+ه+ه+$", r"^ا+ا+$"
    ]
    COMBINATIONS = [
        f"{s0[:-1]}-{s1[1:]}" for s0, s1 in itertools.combinations(AR_STOPWORDS, 2)
    ]
    
    filtered = [
        token for token in tokens
        if not any(re.fullmatch(pat, token) for pat in AR_STOPWORDS + COMBINATIONS)
    ]

    return [t for t in filtered if t != "-" and not t.endswith("-")]

def remove_tags(text):
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"\(?\)", "", text)
    text = re.sub(r"[\[\]()]", "", text)
    return text

def is_punct(s):
    return all(c in "؟،.!…؛" + string.punctuation for c in s)

# Arabic normalization and cleaning
def normalize_text(text):
    text = remove_tags(text)
    text = normalize_unicode(text)
    text = normalize_alef_ar(text)
    text = normalize_alef_maksura_ar(text)
    text = normalize_teh_marbuta_ar(text)

    tokens = simple_word_tokenize(text)
    tokens = remove_asr_errors(tokens)

    try:
        first_valid = next(i for i, t in enumerate(tokens) if not is_punct(t))
        tokens = tokens[first_valid:]
    except StopIteration:
        tokens = []

    if tokens:
        tokens = [t for i, t in enumerate(tokens)
                  if i == 0 or t != tokens[i - 1]]
        tokens = [t for i, t in enumerate(tokens)
                  if i == len(tokens) - 1 or not (is_punct(t) and is_punct(tokens[i + 1]))]

        if not is_punct(tokens[-1][-1]):
            tokens.append(".")

    return " ".join(tokens)

def preprocess_transcript(transcript):
    roles, utterances = [], []

    for line in transcript:
        norm_utts = [normalize_text(u) for u in line["utterance"]]
        combined = " ".join(u for u in norm_utts if u)
        if combined:
            roles.append(line["role"])
            utterances.append(combined)

    assert len(roles) == len(utterances)
    return {"roles": roles, "utterances": utterances}

def preprocess_transcripts(transcripts):
    preprocessed = {}
    for meeting_id, transcript in transcripts.items():
        parsed = parse_transcript_by_speaker(transcript)
        preprocessed[meeting_id] = preprocess_transcript(parsed)
    return preprocessed


In [ ]:
parsed_transcript = parse_transcript_by_speaker(lines)
input_preprocessed = preprocess_transcript(parsed_transcript)


In [ ]:
transcript

In [ ]:
input_preprocessed

In [ ]:
def save_preprocessed(preprocessed):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    with open(os.path.join(OUTPUT_DIR, f"{OUTPUT_FILE}.json"), "w") as f:
        json.dump(preprocessed, f, ensure_ascii=False, indent=4)

In [ ]:
save_preprocessed(input_preprocessed)


In [ ]:
# !zip -r preprocessed_data.zip /content/preprocessed_data


In [ ]:
# -------------------------HERE------------------------
# Run this for me
INPUT_DATA_PATH = "/kaggle/working/preprocessed_data/preprocessed_trunscript.json"
OUTPUT_DATA_PATH = "/kaggle/working/output"

# Run this for you

# BASE_PATH = os.getcwd()
# INPUT_DATA_PATH = os.path.join(BASE_PATH, 'preprocessed_data', 'preprocessed_trunscript.json')
# OUTPUT_DATA_PATH = os.path.join(BASE_PATH, 'output')
# -----------------------------------------------------

# MODEL_SHORT_NAME = "bart_large_xsum_samsum"
# MODEL_SHORT_NAME = "MEETING_SUMMARY"
MODEL_SHORT_NAME = "bart-large-cnn-samsum"
ARABIC_MODEL = "hazemOmrann14/AraBART-summ-finetuned-xsum-finetuned-xsum-finetuned-xsum"

# MODEL = f"facebook/bart-large-xsum"
# MODEL = f"knkarthick/{MODEL_SHORT_NAME}"
MODEL = f"philschmid/{MODEL_SHORT_NAME}"

# SUMMARIZER_MODEL = f"models/{MODEL_SHORT_NAME}/checkpoint-5500"
SUMMARIZER_MODEL = ARABIC_MODEL

In [ ]:
import torch
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

import datetime
import json
import os
import re
import numpy as np
import pandas as pd
import networkx as nx
import math
import nltk
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
if torch.cuda.is_available():
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name())
    torch.cuda.set_device(0)
else:
    print('No GPU available, using the CPU instead.')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
device = 0 if torch.cuda.is_available() else None
summarizer = pipeline("summarization", model=SUMMARIZER_MODEL, device=device)

In [ ]:
def load_preprocessed_transcripts(file_name):
  with open(f"{file_name}", "r") as f:
    preprocessed_transcripts = json.load(f)

  return preprocessed_transcripts

In [ ]:
input_preprocessed = load_preprocessed_transcripts(INPUT_DATA_PATH)
# europarl_preprocessed = load_preprocessed_transcripts(os.path.join(PREPROCESSED_DIR, EUROPARL_DATA_PATH))

In [ ]:
input_preprocessed

In [ ]:
def segment_transcript(max_input_length, transcript, tokenizer):
  def split_line(line, role, tokenizer):
    splits = []

    sentences = sent_tokenize(line)
    split_idx = len(sentences)//2
    line1 = " ".join(sentences[:split_idx]) + '.\n'
    line2 = role + ": " + " ".join(sentences[split_idx:])

    for line in [line1, line2]:
      if len(tokenizer.encode(line)) >= max_input_length:
        splits += split_line(line, role, tokenizer)
      else:
        splits.append(line)

    return splits

  roles = transcript['roles']
  attendees = sorted(list(set(roles)))
  utterances = transcript['utterances']
  segmented_transcript = [""]

  for role, utterance in zip(roles, utterances):
    line = role + ': ' + utterance + '\n'
    # TODO remove short lines?
    tokenized_line = tokenizer.encode(line)

    if len(tokenized_line)>=max_input_length:
        line_splits = split_line(line, role, tokenizer)
    else:
        line_splits = [line]

    for line_split in line_splits:
        tokenized = tokenizer.encode(segmented_transcript[-1]+line_split)
        if len(tokenized)>=max_input_length:
            segmented_transcript.append(line_split)
        else:
            segmented_transcript[-1] += line_split

  return segmented_transcript, attendees

In [ ]:

segmented_transcript_long, attendees_long = segment_transcript(512, input_preprocessed, tokenizer)
segmented_transcript_avg, _ = segment_transcript(768, input_preprocessed, tokenizer)
segmented_transcript_short, _ = segment_transcript(1024, input_preprocessed, tokenizer)

print(segmented_transcript_short[0])

In [ ]:
print(len(segmented_transcript_short))
print(len(segmented_transcript_avg))
print(len(segmented_transcript_long))

In [ ]:
def summarize(input_text, summarizer):
  summarization = summarizer(input_text)[0]["summary_text"].strip()
  return summarization

def generate_summary(segmented_transcript, summarizer):
  summarized_segments = [summarize(transcript_segment, summarizer) for transcript_segment in segmented_transcript]
  return " ".join(summarized_segments)

In [ ]:
# summary_short = generate_summary(segmented_transcript_short, summarizer)
# summary_short

In [ ]:
# summary_avg = generate_summary(segmented_transcript_avg, summarizer)
# summary_avg

In [ ]:
# -------------HERE-----------------
summary_long = generate_summary(segmented_transcript_long, summarizer)
print(summary_long)

# ----------------------------------

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

In [ ]:
from nltk.tokenize import sent_tokenize
import datetime

def fix_entities(text):
    entity_map = {
        "Speaker": "SPEAKER",
        "Organization": "ORGANIZATION",
        "Project": "PROJECT",
        "Location": "LOCATION",
        "Annotator": "ANNOTATOR",
        "Url": "URL",
        "Number": "NUMBER",
        "Password": "PASSWORD",
        "Phone": "PHONE",
        "Path": "PATH",
        "Email": "EMAIL",
        "Other": "OTHER"
    }

    for key, val in entity_map.items():
        pattern = re.compile(rf"{key}\s*(\d+)", re.IGNORECASE)
        text = pattern.sub(lambda m: f"{val}{m.group(1)}", text)

    return text


def create_minutes(summary):
    summary = fix_entities(summary.strip())
    if not summary:
        return "- No summary available."
    sentences = sent_tokenize(summary)
    minutes = "\n".join([f"- {sent}" for sent in sentences])
    return minutes


def format_minutes(attendees, minutes):
    tday = datetime.date.today().strftime("%B %d, %Y")  # e.g. March 28, 2025
    att = ", ".join(attendees) if attendees else "N/A"
    return f"""DATE: {tday}
ATTENDEES: {att}


SUMMARY
{minutes}


Minuted by: AUTOMIN"""


In [ ]:
# ----------------------HERE---------------------
minutes_text = format_minutes(attendees_long, create_minutes(summary_long))

print(minutes_text)

# -----------------------------------------------

# Deploy as an API using FastAPI + ngrok

In [ ]:
!pip install fastapi uvicorn pyngrok

In [ ]:
from fastapi import FastAPI,File, UploadFile, Form
import uvicorn
from pyngrok import ngrok
import nest_asyncio
from pydantic import BaseModel

In [ ]:
nest_asyncio.apply()

In [ ]:
!ngrok authtoken 2v6HSyefAGrtn5M23OyJHPLbwZV_3bcw9gHUCCbfA55k4ZuNT

In [ ]:
app = FastAPI(title="Transcript Processing API")

In [ ]:
# Define a request model to accept the transcript text
class TranscriptRequest(BaseModel):
    transcript: str

# Define a response model for both summary and meeting minutes
class SummaryMinutesResponse(BaseModel):
    summary: str
    minutes: str

@app.post("/summaryMinutes", response_model=SummaryMinutesResponse)
def process_transcript(request: TranscriptRequest):
    print(f"Transcript Content: {transcript[:10]}")
    
    transcript_lines = request.transcript.splitlines()
    
    parsed_transcript = parse_transcript_by_speaker(transcript_lines)
    
    preprocessed = preprocess_transcript(parsed_transcript)
    
    segmented_transcript, attendees = segment_transcript(512, preprocessed, tokenizer)
    
    summary_text = generate_summary(segmented_transcript, summarizer)
    
    minutes_text = format_minutes(attendees, create_minutes(summary_text))

    response = SummaryMinutesResponse(summary=summary_text, minutes=minutes_text)

    print("=== API Output ===")
    print(f"Summary:\n{response.summary}\n")
    print(f"Minutes:\n{response.minutes}\n")
    
    # Return the summary and minutes as a JSON response
    return response

In [ ]:
public_url = ngrok.connect(8000).public_url
print(f"Public API URL: {public_url}")
uvicorn.run(app, host="0.0.0.0", port=8000)